In [1]:
import numpy as np
import random
from PIL import Image
im = Image.open('elder.jpg').convert('L')
im.show()
X = np.array(im)
d, n = X.shape

In [2]:
k, alpha = d // 10, 0.3
S = X.T[random.sample(range(n), int(alpha * n))]

In [3]:
u, s, v = np.linalg.svd(X)
Xk = np.dot(u[:, :k], np.diag(s[:k])).dot(v[:k, :])
print('TSVD:')
print('Squared Error:', np.sum((X - Xk) ** 2))
Image.fromarray(Xk).show()

TSVD:
Squared Error: 1712336.3186744584


In [57]:
p, m_max = k // 2, 9
XR = [X.dot(np.random.normal(loc=0.0, scale=1, size=(n, m_max))) for _ in range(k + p)]
XS = [i ** 2 for i in XR]
for i in range(k + p):
    XS[i] += X.dot(np.random.normal(loc=0.0, scale=1, size=(n, m_max))) ** 2
    XS[i] = np.sqrt(XS[i])

In [47]:
#RSVD
Q, QT = np.zeros((d, 0)), np.zeros((0, d))
for i in range(k + p):
    r = XR[i][:,0]
    r = r - Q.dot(QT.dot(r))
    r = r / np.sqrt(np.sum(r ** 2))
    Q, QT = np.c_[Q, r], np.r_[QT, r.reshape(1, -1)]
u, s, v = np.linalg.svd(QT.dot(X))
Xk = Q.dot(np.dot(u[:, :k], np.diag(s[:k])).dot(v[:k, :]))
print('RSVD:')
print('Squared Error:', np.sum((X - Xk) ** 2))
Image.fromarray(Xk).show()

RSVD:
Squared Error: 3256180.35286838


In [61]:
print('MRSVD:')
for m in range(1, m_max + 1):
    Q, QT = np.zeros((d, 0)), np.zeros((0, d))
    for i in range(k + p):
        r = XS[i][:, :m]
        r = r - Q.dot(QT.dot(r))
        r = r / np.sqrt(np.sum(r ** 2, axis = 0))
        r = r[:, np.argmax(np.sum(S.dot(r) ** 2, axis = 0))]
        Q, QT = np.c_[Q, r], np.r_[QT, r.reshape(1, -1)]
    u, s, v = np.linalg.svd(QT.dot(X))
    Xk = Q.dot(np.dot(u[:, :k], np.diag(s[:k])).dot(v[:k, :]))
    print('t = ', m)
    print('Squared Error:', np.sum((X - Xk) ** 2))
Image.fromarray(Xk).show()

MRSVD:
t =  1
Squared Error: 4039858.0809650593
t =  2
Squared Error: 3147272.9663205426
t =  3
Squared Error: 3254434.5383791327
t =  4
Squared Error: 2963505.658683114
t =  5
Squared Error: 2907612.8688382395
t =  6
Squared Error: 2765283.1994445394
t =  7
Squared Error: 2755148.8433170957
t =  8
Squared Error: 2703270.496020624
t =  9
Squared Error: 2560305.0784847336
